<a href="https://colab.research.google.com/github/Moukthika1253/NaiveBayesClassifier-Rotten-Tomato-reviews/blob/main/NBC_Rotten_tomato_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
df=pd.read_csv('/content/rt_reviews.csv',encoding='latin-1')
X=df.drop(columns=['Freshness']).copy()
y=df['Freshness']
X_train,X_remaining, y_train, y_remaining= train_test_split(X,y,train_size=0.5)
X_test, X_dev, y_test, y_dev = train_test_split(X_remaining,y_remaining,test_size=0.3)
print(f"X_train shape {X_train.shape}, y_train shape {y_train.shape}")
print(f"X_development shape {X_dev.shape}, y_development shape {y_dev.shape}")
print(f"X_test shape {X_test.shape}, y_test shape {y_test.shape}")


X_train shape (240000, 1), y_train shape (240000,)
X_development shape (72000, 1), y_development shape (72000,)
X_test shape (168000, 1), y_test shape (168000,)


In [3]:
import string
import nltk
from nltk import word_tokenize
nltk.download('punkt')
lower_list = []
for i in X_train['Review']:
   lower_list.extend(word_tokenize("".join(j for j in str.lower(i) if j not in string.punctuation)))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
lower_list


['shorn',
 'of',
 'political',
 'subtexts',
 'and',
 'vacuous',
 'in',
 'terms',
 'of',
 'its',
 'characterisation',
 'cores',
 'point',
 'break',
 'is',
 'in',
 'freefall',
 'from',
 'start',
 'to',
 'finish',
 'leaving',
 'the',
 'stunts',
 'looking',
 'like',
 'nothing',
 'more',
 'than',
 'pumpedup',
 'adverts',
 'for',
 'energy',
 'drinks',
 'despite',
 'the',
 'farcical',
 'setups',
 'involving',
 'adjoining',
 'hotel',
 'rooms',
 'private',
 'detectives',
 'and',
 'a',
 'brusque',
 'busybody',
 'therapist',
 'played',
 'with',
 'aching',
 'exaggeration',
 'by',
 'jennifer',
 'aniston',
 'theres',
 'very',
 'little',
 'here',
 'that',
 'brings',
 'the',
 'funny',
 'the',
 'film',
 'veers',
 'quickly',
 'into',
 'predictable',
 'territory',
 'and',
 'doesnt',
 'delve',
 'deeply',
 'enough',
 'into',
 'any',
 'of',
 'its',
 'more',
 'serious',
 'relationship',
 'issues',
 'there',
 'are',
 'some',
 'beautiful',
 'moments',
 'and',
 'some',
 'decent',
 'performances',
 'but',
 'its'

**Calculating word frequencies**

In [6]:
word_dic={}
for i in lower_list:
  if i in word_dic:
    word_dic[i]+=1
  else:
    word_dic[i]=1

In [8]:
word_dic

{'shorn': 31,
 'of': 154681,
 'political': 1340,
 'subtexts': 18,
 'and': 157384,
 'vacuous': 66,
 'in': 76187,
 'terms': 704,
 'its': 64425,
 'characterisation': 100,
 'cores': 6,
 'point': 2321,
 'break': 433,
 'is': 95366,
 'freefall': 5,
 'from': 18065,
 'start': 1110,
 'to': 113717,
 'finish': 437,
 'leaving': 513,
 'the': 272502,
 'stunts': 200,
 'looking': 1301,
 'like': 17429,
 'nothing': 3658,
 'more': 19113,
 'than': 16487,
 'pumpedup': 19,
 'adverts': 4,
 'for': 40360,
 'energy': 1115,
 'drinks': 27,
 'despite': 3031,
 'farcical': 77,
 'setups': 51,
 'involving': 505,
 'adjoining': 3,
 'hotel': 205,
 'rooms': 65,
 'private': 213,
 'detectives': 15,
 'a': 188493,
 'brusque': 7,
 'busybody': 1,
 'therapist': 13,
 'played': 785,
 'with': 44130,
 'aching': 57,
 'exaggeration': 21,
 'by': 19376,
 'jennifer': 264,
 'aniston': 103,
 'theres': 6189,
 'very': 6295,
 'little': 7106,
 'here': 4897,
 'that': 62515,
 'brings': 883,
 'funny': 5376,
 'film': 35520,
 'veers': 153,
 'quickly

**Removing rare words**

In [15]:
word_dic['by']

19376

In [21]:
word_dic_copy=word_dic.copy()
for i,j in word_dic_copy.items():
  if (j) <5:
    del(word_dic[i])

In [ ]:
categories=["Fresh","Rotten"]